In [83]:
import numpy as np
from scipy.signal import convolve2d 
from os import path, getcwd, system, getenv
from astropy.io import fits
from time import perf_counter
import urllib.request
import os

import matplotlib
import math
import matplotlib.pyplot as plt
from matplotlib import cm
import glob
import matplotlib.gridspec as gridspec

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.backends.backend_pdf import PdfPages

import sys,inspect
currentdir = path.dirname(path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = path.dirname(currentdir)
sys.path.append(parentdir)
# from deconv import cube_deconv
# from ppxf_wrap import ppxf_wrap

# import warnings

# if True:
#     print('Set warnings.filterwarnings as "ignore"')
#     print('Warning Message will not be printed')
#     warnings.filterwarnings("ignore")

from astropy.convolution import convolve, Gaussian1DKernel, Box1DKernel
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from scipy import constants
import pandas as pd

# import plotly.express as px
from astroquery.mast import Observations


from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
from astropy.nddata.utils import Cutout2D

from astropy.wcs import WCS
import astropy

from urllib.error import HTTPError
from astroquery.skyview import SkyView
import pickle


computer = 1
maindir = "C:/Users/maxri/Desktop/Classes 4-1/Research/fuse"
obn = 89
skydowni ="./images_redOb{}".format(obn); 

In [84]:
#Create Object List/
#Save to Object Tables

catalogdir = "C:/Users/maxri/Desktop/Classes 4-1/Research/data/VI_129_fuse.dat.fits"
catalog = fits.open(catalogdir)

catheader = catalog[0]; catlist = catalog[1];

dflist = pd.DataFrame(catlist.data)



catalog.close()


obs=dflist.query("Class == {} and Avail=='y'".format(obn));

dflist



obsn1 = []
for i in obs.iterrows():
    b = [i[1][1].split()[0],i[1][2], i[1][3]] 
    obsn1.append(b)
    

    
obsn1

s1 = maindir+"/tbls/Objects{}.tbl".format(obn); 

#Reading Index
indexdir = f"{maindir}/tbls/{obn}index.txt"
indexdir

ipd = pd.read_table(indexdir, header = None)

obsn1




[['NGC1705', 73.5562, -53.3609],
 ['NGC1705', 73.5562, -53.3609],
 ['NGC3310', 159.69, 53.5017],
 ['NGC4214', 183.914, 36.3264],
 ['IRAS19245-4140', 291.992, -41.5744],
 ['NGC7714', 354.058, 2.1553],
 ['NGC7714', 354.059, 2.1552],
 ['M83', 204.252, -29.8668],
 ['NGC5253', 204.983, -31.6408],
 ['MRK357', 20.6675, 23.1694],
 ['NGC3504', 165.797, 27.9725],
 ['NGC3690', 172.129, 58.5614],
 ['NGC3690', 172.129, 58.5614],
 ['IIZW70', 222.735, 35.5717],
 ['NGC4670', 191.32, 27.125],
 ['NGC4670', 191.32, 27.125],
 ['NGC4670', 191.32, 27.125],
 ['POX36', 179.743, -19.0281],
 ['IRAS08339+6517', 129.597, 65.1211],
 ['NGC7714', 354.059, 2.1553],
 ['NGC5430', 210.191, 59.3289],
 ['M82-C', 149.013, 69.6467],
 ['M82-B', 149.0, 69.6547],
 ['M82-A', 148.988, 69.6637],
 ['M82-D', 148.968, 69.7151],
 ['IRAS08339+6517', 129.597, 65.1211],
 ['NGC4194', 183.54, 54.5272],
 ['HARO3', 161.343, 55.9603],
 ['NGC4449-HII', 187.039, 44.0877],
 ['NGC4449-NUC', 187.045, 44.0952],
 ['NGC4194-BKGD', 183.54, 54.5272],


In [201]:
catalogdir2 = "C:/Users/maxri/Desktop/Classes 4-1/Research/data/VII_1B_catalog.dat.fits"
catalog2 = fits.open(catalogdir2)

catheader2 = catalog2[0]; catlist2 = catalog2[1];

dflist2 = pd.DataFrame(catlist2.data)

dflist2

t6 = dflist2[dflist2["NGC"] == 1705].values[0]

dflist2[dflist2["NGC"] == 1705]

rat = t6[3]*15 + t6[4]/4;
if t6[5] == "-":
    dect = (t6[6]+t6[7]/60) * (-1)
else:
    dect = (t6[6]+t6[7]/60) 

print(rat,dect)
dflist2[dflist2["NGC"] == 1705]

73.425 -53.38333333333333


,NGC,m_NGC,Type,RAh,RAm,DE-,DEd,DEm,GLON,GLAT,Xpos,Ypos,Mag,r_Mag,OldDesc,NewDesc,Notes
1747,1705,,5,4,53.7,-,53,23,261.07,-38.73,0,0,13.0,1,"PF,S,R,PMBM",...,*


In [ ]:
minsep = 0.5
pairs = []
for i in range(len(obsn1)):
    if ipd[0][i] == 0:
        continue
    c1 = SkyCoord(obsn1[i][1],obsn1[i][2], unit=u.deg)
    for j in dflist2.iterrows():
        ra2 = j[1][3]*15 + j[1][4]/4; 
        if j[1][5] =="-":
            dec2 = (j[1][6]+j[1][7]/60) * (-1)
        else:
            dec2 = j[1][6]+j[1][7]/60
        c2 = SkyCoord(ra2,dec2, unit = u.deg)
        if c1.separation(c2).deg < minsep:
            pairs.append([f"Fuse{obsn1[i][0]}",f"NGC{j[1][0]} Type{j[1][2]}"])
            print("Pair found")
pairs

Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found


In [94]:
minsep = 0.1
pairs = []
for i in range(len(obsn1)):
    if ipd[0][i] == 0:
        continue
    c1 = SkyCoord(obsn1[i][1],obsn1[i][2], unit=u.deg)
    clist.query("0")
            
pairs

[]